In [6]:
!pip -q install requests python-dotenv

In [7]:
%%writefile run.py
#!/usr/bin/env python3
import os, sys, csv, requests, argparse
from collections import deque
from pathlib import Path
from dotenv import load_dotenv

def norm(s: str) -> str:
    return (s or "").strip().lower()

def build_headers_and_cookies(env_path: str):
    load_dotenv(dotenv_path=env_path)
    bearer = (os.getenv("BEARER_TOKEN") or "").strip()
    session = (os.getenv("SESSION_COOKIE") or "").strip()
    headers = {"Accept": "application/json", "User-Agent": "python-requests/2.31"}
    if bearer:
        headers["Authorization"] = f"Bearer {bearer}"
    cookies = {"session": session} if session else {}
    return headers, cookies

def fetch_tree(base_url: str, tenant_id: str, headers, cookies, timeout: int):
    url = f"{base_url.rstrip('/')}/api/v2/tenants/{tenant_id}/hierarchy-tree"
    resp = requests.get(url, headers=headers, cookies=cookies, timeout=timeout)
    resp.raise_for_status()
    mapping = resp.json().get("mapping", {}) or {}
    nodes = {az: (n if "azId" in n else {**n, "azId": az}) for az, n in mapping.items()}
    return nodes

def find_client(nodes, client_name):
    client_name_n = norm(client_name)
    matches = [
        n for n in nodes.values()
        if str(n.get("type", "")).upper() == "CLIENT" and norm(n.get("name")) == client_name_n
    ]
    if not matches:
        return None
    matches.sort(key=lambda n: n.get("azId"))
    return matches[0]

def bfs_collect(nodes, start_ids, type_name=None, name=None):
    want_type = type_name.upper() if type_name else None
    want_name = norm(name) if name else None
    seen, out = set(), []
    q = deque(start_ids or [])
    while q:
        az = q.popleft()
        if az in seen: continue
        seen.add(az)
        node = nodes.get(az)
        if not node: continue
        t_ok = True if not want_type else (str(node.get("type", "")).upper() == want_type)
        n_ok = True if not want_name else (norm(node.get("name")) == want_name)
        if t_ok and n_ok:
            out.append(node)
        for child in (node.get("children") or []):
            q.append(child)
    return out

def brand_has_category(brand_node, category_name):
    if not category_name: return True
    cat_n = norm(category_name)
    for c in (brand_node.get("categories") or []):
        if norm(c.get("name")) == cat_n:
            return True
    return False

def resolve_brand_azid(nodes, market_name, client_name, brand_name, category_name):
    client = find_client(nodes, client_name)
    if not client:
        raise ValueError(f"Client not found: {client_name}")
    markets = bfs_collect(nodes, client.get("children") or [], type_name="MARKET", name=market_name)
    if not markets:
        raise ValueError(f"Market '{market_name}' not found under client '{client_name}'")
    candidates = []
    for market in sorted(markets, key=lambda n: n.get("azId")):
        brands = bfs_collect(nodes, market.get("children") or [], type_name="BRAND", name=brand_name)
        brands = [b for b in brands if brand_has_category(b, category_name)]
        candidates.extend(brands)
    if not candidates:
        raise ValueError(
            f"Brand '{brand_name}' with category '{category_name}' not found under market '{market_name}' (client '{client_name}')"
        )
    candidates.sort(key=lambda n: n.get("azId"))
    return candidates[0]["azId"]

def read_csv_rows(path):
    required = {"Market", "Client", "Brand", "Category"}
    rows = []
    with open(path, "r", encoding="utf-8-sig", newline="") as f:
        reader = csv.DictReader(f)
        if not reader.fieldnames:
            raise ValueError("CSV has no header")
        missing = required - set(reader.fieldnames)
        if missing:
            raise ValueError(
                f"CSV must have header columns: {', '.join(sorted(required))}. Missing: {', '.join(sorted(missing))}"
            )
        for r in reader:
            market = (r.get("Market") or "").strip()
            client = (r.get("Client") or "").strip()
            brand = (r.get("Brand") or "").strip()
            category = (r.get("Category") or "").strip()
            if market and client and brand and category:
                rows.append((market, client, brand, category))
    return rows

def write_csv(path: Path, rows, fieldnames):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        if rows:
            writer.writerows(rows)

def main():
    parser = argparse.ArgumentParser(description="Resolve brand azIds and export success/failed CSVs.")
    parser.add_argument("-i", "--input", required=True, help="Path to input CSV (Market,Client,Brand,Category)")
    parser.add_argument("-e", "--env", default="auth.env", help="Path to .env with BEARER_TOKEN / SESSION_COOKIE")
    parser.add_argument("-o", "--output-dir", default="outputs", help="Directory to write output files")
    parser.add_argument("-t", "--tenant-id", default="4c039217-7d17-4207-8314-98348983718a", help="Tenant ID")
    parser.add_argument("--base-url", default="https://media.os.wpp.com", help="Base API URL")
    parser.add_argument("--timeout", type=int, default=30, help="HTTP timeout in seconds")
    args = parser.parse_args()

    output_dir = Path(args.output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    output_txt = output_dir / "output.txt"

    headers, cookies = build_headers_and_cookies(args.env)
    try:
        nodes = fetch_tree(args.base_url, args.tenant_id, headers, cookies, args.timeout)
    except Exception as e:
        print(f"Request failed: {e}", file=sys.stderr)
        return 1

    try:
        rows = read_csv_rows(args.input)
    except Exception as e:
        print(f"Input error: {e}", file=sys.stderr)
        return 1

    success_list, failed_list = [], []

    with open(output_txt, "w", encoding="utf-8") as logf:
        for market, client, brand, category in rows:
            try:
                azid = resolve_brand_azid(nodes, market, client, brand, category)
                line = f"({market},{client},{brand},{category}): {azid}"
                print(line)
                logf.write(line + "\n"); logf.flush()
                success_list.append(
                    {"market": market, "client": client, "brand": brand, "category": category, "uuid": azid}
                )
            except Exception as e:
                line = f"Error ({market},{client},{brand},{category}): {e}"
                print(line, file=sys.stderr)
                logf.write(line + "\n"); logf.flush()
                failed_list.append(
                    {"market": market, "client": client, "brand": brand, "category": category, "error_message": str(e)}
                )

    success_list.sort(key=lambda x: (x["client"], x["market"], x["brand"]))
    failed_list.sort(key=lambda x: (x["client"], x["market"], x["brand"]))
    write_csv(output_dir / "success.csv", success_list, ["market", "client", "brand", "category", "uuid"])
    write_csv(output_dir / "failed.csv", failed_list, ["market", "client", "brand", "category", "error_message"])

    print(f"✅ Done. Success: {len(success_list)} | Failed: {len(failed_list)}")
    print(f"📄 Outputs in: {output_dir.resolve()}")
    return 0

if __name__ == "__main__":
    sys.exit(main())

Overwriting run.py


In [8]:
from getpass import getpass

bearer = getpass("Enter BEARER_TOKEN (or leave blank): ")
session = getpass("Enter SESSION_COOKIE (or leave blank): ")

with open("auth.env", "w", encoding="utf-8") as f:
    f.write(f"BEARER_TOKEN={bearer}\nSESSION_COOKIE={session}\n")

print("auth.env written (values hidden).")

Enter BEARER_TOKEN (or leave blank): ··········
Enter SESSION_COOKIE (or leave blank): ··········
auth.env written (values hidden).


In [9]:
from google.colab import files
uploaded = files.upload()  # upload your input CSV
input_path = next(iter(uploaded))  # take first uploaded filename
print("Using:", input_path)

Saving input.csv to input (1).csv
Using: input (1).csv


In [11]:
tenant_id = "4c039217-7d17-4207-8314-98348983718a"
base_url  = "https://media.os.wpp.com"
timeout_s = 30

import subprocess
cmd = [
    "python", "run.py",
    "-i", input_path,
    "-e", "auth.env",
    "-o", "outputs",
    "-t", tenant_id,
    "--base-url", base_url,
    "--timeout", str(timeout_s),
]
print("Running:", " ".join(cmd))
subprocess.run(cmd, check=False)

Running: python run.py -i input (1).csv -e auth.env -o outputs -t 4c039217-7d17-4207-8314-98348983718a --base-url https://media.os.wpp.com --timeout 30


CompletedProcess(args=['python', 'run.py', '-i', 'input (1).csv', '-e', 'auth.env', '-o', 'outputs', '-t', '4c039217-7d17-4207-8314-98348983718a', '--base-url', 'https://media.os.wpp.com', '--timeout', '30'], returncode=0)

In [12]:
from google.colab import files
!zip -qr outputs.zip outputs
files.download("outputs.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>